In [1]:
from IPython.display import HTML

html_content = """
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
<style>
    body {
        background-image: linear-gradient(rgba(42, 82, 152, 1), rgba(94, 198, 204, 1));
        min-height: 100vh;
    }
    /* Or target a specific div */
    .gradient-bg {
        background-image: linear-gradient(rgba(42, 82, 152, 1), rgba(94, 198, 204, 1));
        padding: 20px;
    }
</style>
<div class="gradient-bg">
    DevPost Hackathon 2024-2025
</div>
"""

display(HTML(html_content))


import os  # Example import - replace with actual required imports

### **Overview of Enhancements**

1. **Movement System:**
   - **Dice Roll or Card Draw:** Players can choose to roll a die or draw a card to determine their movement.
   - **Region and Availability Zone (AZ) Selection:** Players land on different regions and AZs based on their movement.
   - **Home Region:** Each player has a designated home region with specific advantages.

2. **Serverless Spells:**
   - **Quick-Use Abilities:** Represent serverless services (e.g., AWS Lambda, CloudFront) as spells that provide quick advantages.
   - **Spell Mechanics:** Spells can be cast during encounters to influence outcomes.

3. **Turn-Based Setup and Threat Encounters:**
   - **Setup Phase:** At the beginning of each turn, players choose security measures.
   - **Threat Encounter:** Players face threats based on the region they've landed in.
   - **Charges and Constraints:** Operating in non-home regions incurs charges, affecting resource management.

### **Step-by-Step Integration**

We'll tackle each enhancement one by one. Ensure that each snippet is placed appropriately within your existing codebase.

---



## **1. Movement System**

### **a. Define the Game Board and Regions**

First, let's define a `GameBoard` class that contains different regions and their Availability Zones (AZs). This class will manage the regions where players can land.

```python
# game_board.py

from dataclasses import dataclass
from typing import List

@dataclass
class AvailabilityZone:
    """
    Represents an AWS Availability Zone within a region.
    """
    name: str
    bonus: Optional[str] = None  # Optional bonus attribute

@dataclass
class Region:
    """
    Represents an AWS Region containing multiple Availability Zones.
    """
    name: str
    azs: List[AvailabilityZone]
    special_access_required: bool = False  # Indicates if special access is needed

class GameBoard:
    """
    Represents the game board with all regions and their AZs.
    """
    def __init__(self):
        self.regions: List[Region] = self.create_regions()

    def create_regions(self) -> List[Region]:
        """
        Initializes and returns a list of regions with their AZs.
        """
        regions = [
            Region(
                name="US East (N. Virginia)",
                azs=[
                    AvailabilityZone(name="us-east-1a", bonus="Agility"),
                    AvailabilityZone(name="us-east-1b", bonus="Elasticity"),
                    AvailabilityZone(name="us-east-1c"),
                ]
            ),
            Region(
                name="US West (Oregon)",
                azs=[
                    AvailabilityZone(name="us-west-2a", bonus="Resilience"),
                    AvailabilityZone(name="us-west-2b"),
                    AvailabilityZone(name="us-west-2c", bonus="Fault Tolerance"),
                ]
            ),
            Region(
                name="Special Region",
                azs=[
                    AvailabilityZone(name="special-az-1", bonus="Availability"),
                ],
                special_access_required=True
            ),
            # Add more regions and AZs as needed
        ]
        return regions
```

**Integration Steps:**

1. **Create a New File:** Save the above code in a new file named `game_board.py`.
2. **Import `GameBoard`:** In your main game file (`game.py`), import the `GameBoard` class.

    ```python
    from game_board import GameBoard, Region, AvailabilityZone
    ```

---

### **b. Implement Player Movement**

Next, we'll create a `Player` class that handles movement across the game board using dice rolls or card draws.

```python
# player.py

import random
from typing import Optional
from game_board import GameBoard, Region, AvailabilityZone

class Player:
    """
    Represents a player in the game.
    """
    def __init__(self, name: str, game_board: GameBoard):
        self.name = name
        self.game_board = game_board
        self.current_position: int = 0  # Index in the game board's regions list
        self.home_region: Optional[Region] = None
        self.home_az: Optional[AvailabilityZone] = None

    def set_home_region(self):
        """
        Sets the player's home region and AZ at the start of the game.
        """
        self.home_region = self.game_board.regions[0]  # For example, first region as home
        self.home_az = self.home_region.azs[0]        # First AZ in home region
        print(f"{self.name}'s home region is {self.home_region.name} - {self.home_az.name}.")

    def roll_dice(self) -> int:
        """
        Simulates rolling a six-sided die to determine movement steps.
        """
        return random.randint(1, 6)

    def draw_card(self) -> int:
        """
        Simulates drawing a card that determines movement steps (1-6).
        """
        return random.randint(1, 6)  # Replace with card logic if needed

    def move(self, steps: int):
        """
        Moves the player a certain number of steps on the game board.
        """
        total_regions = len(self.game_board.regions)
        self.current_position = (self.current_position + steps) % total_regions
        landed_region = self.game_board.regions[self.current_position]
        landed_az = random.choice(landed_region.azs)  # Randomly select an AZ within the region
        print(f"{self.name} landed on {landed_region.name} - {landed_az.name}.")
        return landed_region, landed_az
```

**Integration Steps:**

1. **Create a New File:** Save the above code in a new file named `player.py`.
2. **Import `Player`:** In your main game file (`game.py`), import the `Player` class.

    ```python
    from player import Player
    ```

3. **Initialize Player and Game Board:**

    ```python
    # Inside run_game()

    def run_game():
        # Initialize game board
        game_board = GameBoard()
        
        # Initialize player
        player = Player(name="Player1", game_board=game_board)
        player.set_home_region()
        
        # Continue with the rest of the game loop
    ```

---


## **2. Serverless Spells**

Serverless services like AWS Lambda and CloudFront can be represented as spells that players can cast during encounters for quick advantages.

### **a. Define Serverless Spells**

Create a `ServerlessSpell` class and define specific spells.

```python
# spells.py

import random
from abc import ABC, abstractmethod
from vpc_defender import VPCDefender

class ServerlessSpell(ABC):
    """
    Abstract base class for serverless spells.
    """
    name: str
    description: str

    @abstractmethod
    def cast(self, defender: VPCDefender):
        pass

class LambdaEdgeSpell(ServerlessSpell):
    """
    Represents the AWS Lambda@Edge spell.
    """
    name = "AWS Lambda@Edge"
    description = "Run code closer to users to increase agility."

    def cast(self, defender: VPCDefender):
        bonus = random.randint(5, 15)
        defender.stats.agility += bonus
        print(f"{self.name} cast! Agility increased by {bonus}.")

class CloudFrontSpell(ServerlessSpell):
    """
    Represents the AWS CloudFront spell.
    """
    name = "AWS CloudFront"
    description = "Accelerate content delivery to improve elasticity."

    def cast(self, defender: VPCDefender):
        bonus = random.randint(5, 15)
        defender.stats.elasticity += bonus
        print(f"{self.name} cast! Elasticity increased by {bonus}.")
```

**Integration Steps:**

1. **Create a New File:** Save the above code in a new file named `spells.py`.
2. **Import Spells:** In your main game file (`game.py`), import the spells.

    ```python
    from spells import LambdaEdgeSpell, CloudFrontSpell
    ```

---

### **b. Integrate Spells into `VPCDefender`**

Assuming you have a `VPCDefender` class, add spell-related methods.

```python
# vpc_defender.py

from typing import Dict
from spells import ServerlessSpell, LambdaEdgeSpell, CloudFrontSpell

class VPCDefender:
    """
    Represents the VPC Defender character.
    """
    def __init__(self, name: str, stats: CharacterStats):
        self.name = name
        self.stats = stats
        self.level = 1
        self.max_health = self.calculate_max_health()
        self.current_health = self.max_health
        self.offensive_measures = SecurityMeasures.get_offensive_measures()
        self.defensive_measures = SecurityMeasures.get_defensive_measures()
        self.active_measures: List[SecurityMeasure] = []
        self.preparation_points = 100
        self.resources = Resources(
            compute_points=100,
            network_bandwidth=50,
            storage_capacity=10
        )
        self.status_effects: List[SecurityStatus] = []
        self.inventory: List[SecurityItem] = []
        
        # Initialize spells
        self.spells: Dict[str, ServerlessSpell] = {
            "lambda_edge": LambdaEdgeSpell(),
            "cloudfront": CloudFrontSpell(),
            # Add more spells as needed
        }

    def cast_spell(self, spell_key: str) -> str:
        """
        Casts a serverless spell if available.
        """
        spell = self.spells.get(spell_key)
        if not spell:
            return f"Spell '{spell_key}' not found."
        spell.cast(self)
        return f"Cast spell: {spell.name}."
```

**Integration Steps:**

1. **Update `VPCDefender` Class:**
   - Ensure that the `VPCDefender` class is in a separate file (e.g., `vpc_defender.py`) and is properly imported in `game.py`.
   - Add the `spells` dictionary and the `cast_spell` method as shown above.

2. **Import and Use in Game Loop:**

    ```python
    # Inside run_game()

    from vpc_defender import VPCDefender

    def run_game():
        # Initialize game board and player
        game_board = GameBoard()
        player = Player(name="Player1", game_board=game_board)
        player.set_home_region()
        
        # Initialize defender
        stats = CharacterStats(
            elasticity=70,
            fault_tolerance=65,
            agility=75,
            resilience=80,
            availability=85
        )
        defender = VPCDefender(name="CloudGuardian", stats=stats)
        
        # Assign defender to player (if necessary)
        player.defender = defender

        # Example of casting a spell
        spell_choice = input("Do you want to cast a serverless spell? (yes/no): ").strip().lower()
        if spell_choice == 'yes':
            print("Available Spells:")
            for key, spell in defender.spells.items():
                print(f"- {key}: {spell.description}")
            selected_spell = input("Enter the spell key to cast: ").strip().lower()
            result = defender.cast_spell(selected_spell)
            print(result)
        
        # Continue with the rest of the game loop
    ```

---


## **3. Turn-Based Setup and Threat Encounters**

### **a. Modify the Game Loop to Include Setup and Encounters**

Enhance your main game loop to include the setup phase, movement, and threat encounters.

```python
# game.py

from game_board import GameBoard
from player import Player
from vpc_defender import VPCDefender
from security_measures import SecurityMeasures  # Ensure this is defined
from spells import LambdaEdgeSpell, CloudFrontSpell
from security_battle import SecurityBattle  # Ensure this is defined

def run_game():
    """Runs the main game loop."""
    # Initialize game board
    game_board = GameBoard()
    
    # Initialize player
    player = Player(name="Player1", game_board=game_board)
    player.set_home_region()
    
    # Initialize defender
    stats = CharacterStats(
        elasticity=70,
        fault_tolerance=65,
        agility=75,
        resilience=80,
        availability=85
    )
    defender = VPCDefender(name="CloudGuardian", stats=stats)
    player.defender = defender  # Link defender to player
    
    # Initialize spells (already handled in VPCDefender)
    
    # Start the game loop
    game_over = False
    while not game_over:
        print("\n=== New Turn ===")
        
        # Step 1: Player chooses to roll a die or draw a card to move
        move_choice = input("Do you want to 'roll' the die or 'draw' a card to move? (roll/draw): ").strip().lower()
        if move_choice == 'roll':
            steps = player.roll_dice()
            print(f"You rolled a {steps}.")
        elif move_choice == 'draw':
            steps = player.draw_card()
            print(f"You drew a card and move {steps} steps.")
        else:
            print("Invalid choice. Defaulting to rolling the die.")
            steps = player.roll_dice()
            print(f"You rolled a {steps}.")
        
        # Step 2: Move the player
        landed_region, landed_az = player.move(steps)
        
        # Step 3: Apply region-specific charges if not in home region
        if landed_region != player.home_region:
            charge = 10  # Example charge value
            if defender.preparation_points >= charge:
                defender.preparation_points -= charge
                print(f"Operating in {landed_region.name} incurs a charge of {charge} preparation points.")
            else:
                print("Insufficient preparation points to cover regional charges.")
                # Implement consequences (e.g., penalties)
        
        # Step 4: Setup Phase - Implement security measures
        print("\n--- Setup Phase ---")
        print(f"Preparation Points: {defender.preparation_points}")
        print("Available Defensive Measures:")
        for key, measure in defender.defensive_measures.items():
            print(f"- {key}: {measure.description} (Cost: {measure.cost}, Effectiveness: {measure.effectiveness})")
        print("Available Offensive Measures:")
        for key, measure in defender.offensive_measures.items():
            print(f"- {key}: {measure.description} (Cost: {measure.cost}, Effectiveness: {measure.effectiveness})")
        
        while True:
            setup_action = input("Enter 'measure' to implement a security measure or 'done' to finish setup: ").strip().lower()
            if setup_action == 'measure':
                measure_name = input("Enter the measure key to implement: ").strip().lower()
                result = defender.implement_security_measure(measure_name)
                print(result)
            elif setup_action == 'done':
                if len(defender.active_measures) == 0:
                    print("You must implement at least one security measure before proceeding.")
                else:
                    print("Setup phase completed.")
                    break
            else:
                print("Invalid action. Please enter 'measure' or 'done'.")
        
        # Step 5: Threat Encounter
        print("\n--- Threat Encounter ---")
        threat = generate_threat_for_region(landed_region.name)
        battle = SecurityBattle(defender, threat)
        battle.start_battle()
        
        # Step 6: Check for Game Over Conditions
        if not defender.is_alive():
            print("Your defender has been compromised! Game Over.")
            game_over = True
        elif battle.is_threat_defeated():
            print(f"Successfully defeated the threat: {threat.name}!")
            defender.level_up()
            # Optionally, reset active measures or continue
        else:
            print("The threat remains active.")
        
        # Step 7: Decide to Continue or End Game
        continue_choice = input("Do you want to continue to the next turn? (yes/no): ").strip().lower()
        if continue_choice != 'yes':
            print("Thank you for playing!")
            game_over = True

# Ensure that all necessary functions like generate_threat_for_region and SecurityBattle are properly defined and imported.
```

**Integration Steps:**

1. **Update `run_game` Function:**
   - Incorporate movement choices (`roll` or `draw`).
   - Implement the setup phase where players choose security measures.
   - Handle threat encounters based on the landed region.

2. **Ensure Supporting Functions are Defined:**
   - **`generate_threat_for_region`:** This function generates threats based on the region. Ensure it's defined either in the same file or imported.

    ```python
    # threat_generator.py

    from threat import Threat

    def generate_threat_for_region(region: str) -> Threat:
        """
        Generates a threat based on the region.
        """
        region_threats = {
            "US East (N. Virginia)": Threat(
                name="DDoS Attack",
                attack_type="Network Flood",
                power=25,
                persistence=3,
                adaptability=60
            ),
            "US West (Oregon)": Threat(
                name="Data Breach",
                attack_type="Unauthorized Access",
                power=30,
                persistence=4,
                adaptability=70
            ),
            "Special Region": Threat(
                name="Advanced Threat",
                attack_type="Multi-vector Attack",
                power=35,
                persistence=5,
                adaptability=80
            ),
            # Add more regions and corresponding threats
        }
        return region_threats.get(region, Threat(
            name="Generic Threat",
            attack_type="Unknown",
            power=20,
            persistence=2,
            adaptability=50
        ))
    ```

   - **Import `generate_threat_for_region`:**

    ```python
    from threat_generator import generate_threat_for_region
    ```

3. **Define `SecurityBattle` Class:**
   - Ensure you have a `SecurityBattle` class that handles the battle logic.

    ```python
    # security_battle.py

    class SecurityBattle:
        """
        Manages the battle between the VPC Defender and a Threat.
        """
        def __init__(self, defender: VPCDefender, threat: Threat):
            self.defender = defender
            self.threat = threat
            self.round = 1

        def start_battle(self):
            """
            Starts the battle until either the defender or the threat is defeated.
            """
            print(f"\nA threat '{self.threat.name}' has emerged! Preparing for battle...")
            while self.defender.is_alive() and self.threat.persistence > 0:
                print(f"\n--- Round {self.round} ---")
                self.player_turn()
                if not self.threat_defeated():
                    self.threat_turn()
                self.round += 1
            if self.defender.is_alive() and self.threat.persistence <= 0:
                print("Threat has been neutralized!")
            elif not self.defender.is_alive():
                print("Defender has been defeated!")

        def player_turn(self):
            """
            Handles the defender's actions during their turn.
            """
            action = input("Do you want to 'attack', 'cast spell', or 'use item'? ").strip().lower()
            if action == 'attack':
                offensive_power = sum(
                    measure.effectiveness for measure in self.defender.active_measures
                    if measure.strategy == SecurityStrategy.OFFENSIVE
                )
                counter_damage = int(offensive_power * (1 + self.defender.stats.agility / 100))
                self.threat.power -= counter_damage
                print(f"Attacked the threat! Dealt {counter_damage} damage.")
            elif action == 'cast spell':
                print("Available Spells:")
                for key, spell in self.defender.spells.items():
                    print(f"- {key}: {spell.description}")
                spell_key = input("Enter the spell key to cast: ").strip().lower()
                result = self.defender.cast_spell(spell_key)
                print(result)
            elif action == 'use item':
                # Implement item usage if inventory is still desired
                item_name = input("Enter the name of the item to use: ").strip()
                result = self.defender.use_item(item_name)
                print(result)
            else:
                print("Invalid action. Skipping turn.")
        
        def threat_turn(self):
            """
            Handles the threat's actions during its turn.
            """
            damage = self.threat.calculate_damage(self.defender)
            is_defeated = self.defender.take_damage(damage)
            if is_defeated:
                print("The threat has defeated your defender!")
            else:
                print(f"The threat dealt {damage} damage to your defender.")
        
        def threat_defeated(self) -> bool:
            """
            Checks if the threat has been defeated.
            """
            return self.threat.power <= 0
    ```

   - **Ensure `Threat` Class has `calculate_damage`:**

    ```python
    # threat.py

    @dataclass
    class Threat:
        name: str
        attack_type: str
        power: int
        persistence: int
        adaptability: int

        def calculate_damage(self, defender: VPCDefender) -> int:
            """
            Calculates the damage inflicted by the threat on the defender.
            """
            base_damage = random.randint(self.power - 5, self.power + 5)
            # Apply defender's defensive measures
            for measure in defender.active_measures:
                if measure.strategy == SecurityStrategy.DEFENSIVE:
                    base_damage = max(0, base_damage - measure.effectiveness)
            # Apply defender's stats for further damage reduction
            damage_reduction = (defender.stats.fault_tolerance + defender.stats.resilience) / 100
            final_damage = int(base_damage * (1 - damage_reduction))
            return final_damage
    ```

   - **Import `SecurityBattle`:**

    ```python
    from security_battle import SecurityBattle
    ```

---


## **4. Removing Inventory (Optional)**

If you decide that inventory management is no longer needed, you can simplify the `VPCDefender` class by removing related attributes and methods.

### **a. Update `VPCDefender` Class**

```python
# vpc_defender.py

class VPCDefender:
    """
    Represents the VPC Defender character.
    """
    def __init__(self, name: str, stats: CharacterStats):
        self.name = name
        self.stats = stats
        self.level = 1
        self.max_health = self.calculate_max_health()
        self.current_health = self.max_health
        self.offensive_measures = SecurityMeasures.get_offensive_measures()
        self.defensive_measures = SecurityMeasures.get_defensive_measures()
        self.active_measures: List[SecurityMeasure] = []
        self.preparation_points = 100
        self.resources = Resources(
            compute_points=100,
            network_bandwidth=50,
            storage_capacity=10
        )
        self.status_effects: List[SecurityStatus] = []
        
        # Initialize spells
        self.spells: Dict[str, ServerlessSpell] = {
            "lambda_edge": LambdaEdgeSpell(),
            "cloudfront": CloudFrontSpell(),
            # Add more spells as needed
        }

    # Remove inventory-related attributes and methods
    # Remove 'inventory' attribute
    # Remove 'add_item' and 'use_item' methods
    
    def cast_spell(self, spell_key: str) -> str:
        """
        Casts a serverless spell if available.
        """
        spell = self.spells.get(spell_key)
        if not spell:
            return f"Spell '{spell_key}' not found."
        spell.cast(self)
        return f"Cast spell: {spell.name}."

    # Existing methods...
```

**Integration Steps:**

1. **Remove Inventory Attributes and Methods:**
   - Delete the `inventory` attribute.
   - Remove methods like `add_item` and `use_item` from the `VPCDefender` class.

2. **Update References:**
   - Ensure that no part of your game loop or other classes references the removed inventory functionalities.
   - For example, in `SecurityBattle`, remove or comment out any code that handles item usage.

    ```python
    # security_battle.py

    def player_turn(self):
        """Handles the defender's actions during their turn."""
        print("It's your turn.")
        action = input("Do you want to 'attack', 'cast spell', or 'use item'? ").strip().lower()
        if action == 'attack':
            # Perform attack logic
            pass
        elif action == 'cast spell':
            spell_name = input("Enter the name of the spell to cast: ").strip().lower()
            result = self.defender.cast_spell(spell_name)
            print(result)
        elif action == 'use item':
            # Since inventory is removed, inform the player
            print("No items available to use.")
        else:
            print("Invalid action.")
    ```

---




## **5. Putting It All Together**

Now, let's assemble the key parts into your main game file (`game.py`) with appropriate comments.

```python
# game.py

# Import necessary modules and classes
from game_board import GameBoard
from player import Player
from vpc_defender import VPCDefender
from threat_generator import generate_threat_for_region
from security_battle import SecurityBattle

def run_game():
    """Runs the main game loop."""
    # Initialize game board
    game_board = GameBoard()
    
    # Initialize player
    player = Player(name="Player1", game_board=game_board)
    player.set_home_region()
    
    # Initialize defender with stats
    stats = CharacterStats(
        elasticity=70,
        fault_tolerance=65,
        agility=75,
        resilience=80,
        availability=85
    )
    defender = VPCDefender(name="CloudGuardian", stats=stats)
    player.defender = defender  # Link defender to player
    
    # Initialize spells are handled within VPCDefender
    
    # Start the game loop
    game_over = False
    while not game_over:
        print("\n=== New Turn ===")
        
        # Step 1: Choose movement method
        move_choice = input("Do you want to 'roll' the die or 'draw' a card to move? (roll/draw): ").strip().lower()
        if move_choice == 'roll':
            steps = player.roll_dice()
            print(f"You rolled a {steps}.")
        elif move_choice == 'draw':
            steps = player.draw_card()
            print(f"You drew a card and move {steps} steps.")
        else:
            print("Invalid choice. Defaulting to rolling the die.")
            steps = player.roll_dice()
            print(f"You rolled a {steps}.")
        
        # Step 2: Move the player
        landed_region, landed_az = player.move(steps)
        
        # Step 3: Apply charges if not in home region
        if landed_region != player.home_region:
            charge = 10  # Example charge value
            if defender.preparation_points >= charge:
                defender.preparation_points -= charge
                print(f"Operating in {landed_region.name} incurs a charge of {charge} preparation points.")
            else:
                print("Insufficient preparation points to cover regional charges.")
                # Implement consequences (e.g., penalties)
        
        # Step 4: Setup Phase - Implement security measures
        print("\n--- Setup Phase ---")
        print(f"Preparation Points: {defender.preparation_points}")
        print("Available Defensive Measures:")
        for key, measure in defender.defensive_measures.items():
            print(f"- {key}: {measure.description} (Cost: {measure.cost}, Effectiveness: {measure.effectiveness})")
        print("Available Offensive Measures:")
        for key, measure in defender.offensive_measures.items():
            print(f"- {key}: {measure.description} (Cost: {measure.cost}, Effectiveness: {measure.effectiveness})")
        
        while True:
            setup_action = input("Enter 'measure' to implement a security measure or 'done' to finish setup: ").strip().lower()
            if setup_action == 'measure':
                measure_name = input("Enter the measure key to implement: ").strip().lower()
                result = defender.implement_security_measure(measure_name)
                print(result)
            elif setup_action == 'done':
                if len(defender.active_measures) == 0:
                    print("You must implement at least one security measure before proceeding.")
                else:
                    print("Setup phase completed.")
                    break
            else:
                print("Invalid action. Please enter 'measure' or 'done'.")
        
        # Step 5: Threat Encounter
        print("\n--- Threat Encounter ---")
        threat = generate_threat_for_region(landed_region.name)
        battle = SecurityBattle(defender, threat)
        battle.start_battle()
        
        # Step 6: Check for Game Over Conditions
        if not defender.is_alive():
            print("Your defender has been compromised! Game Over.")
            game_over = True
        elif battle.is_threat_defeated():
            print(f"Successfully defeated the threat: {threat.name}!")
            defender.level_up()
            # Optionally, reset active measures or continue
        else:
            print("The threat remains active.")
        
        # Step 7: Decide to Continue or End Game
        continue_choice = input("Do you want to continue to the next turn? (yes/no): ").strip().lower()
        if continue_choice != 'yes':
            print("Thank you for playing!")
            game_over = True

# Run the game
if __name__ == "__main__":
    run_game()
```

**Key Points and Comments:**

- **Imports:**
  - Ensure all classes (`GameBoard`, `Player`, `VPCDefender`, `SecurityBattle`) and functions (`generate_threat_for_region`) are correctly imported.
  
- **Game Initialization:**
  - Initialize the game board and player.
  - Assign a home region and AZ to the player.
  - Initialize the defender with specific stats.
  
- **Movement Phase:**
  - Players choose to roll a die or draw a card to determine movement.
  - Landing on a new region/AZ is handled, including applying charges for non-home regions.
  
- **Setup Phase:**
  - Players implement security measures based on available preparation points.
  - Defensive and offensive measures are displayed for selection.
  
- **Threat Encounter:**
  - Based on the landed region, a specific threat is generated.
  - The `SecurityBattle` class manages the encounter between the defender and the threat.
  
- **Game Over Conditions:**
  - Check if the defender is still alive or if the threat has been defeated.
  - Level up the defender if a threat is defeated.
  
- **Continuation Prompt:**
  - Players can choose to continue to the next turn or end the game.

---




## **6. Additional Enhancements and Tips**

### **a. Define Security Measures**

Ensure you have a `SecurityMeasures` class or module that defines available security measures.

```python
# security_measures.py

from dataclasses import dataclass
from enum import Enum
from typing import Dict

class SecurityStrategy(Enum):
    OFFENSIVE = "Offensive"
    DEFENSIVE = "Defensive"

@dataclass
class SecurityMeasure:
    name: str
    strategy: SecurityStrategy
    cost: int
    effectiveness: int
    setup_time: int
    description: str

class SecurityMeasures:
    """
    Provides available offensive and defensive security measures.
    """
    @staticmethod
    def get_offensive_measures() -> Dict[str, SecurityMeasure]:
        return {
            "penetration_test": SecurityMeasure(
                name="Penetration Testing",
                strategy=SecurityStrategy.OFFENSIVE,
                cost=30,
                effectiveness=25,
                setup_time=3,
                description="Proactively identify vulnerabilities through penetration testing."
            ),
            "security_scanning": SecurityMeasure(
                name="Security Scanning",
                strategy=SecurityStrategy.OFFENSIVE,
                cost=15,
                effectiveness=20,
                setup_time=1,
                description="Scan infrastructure for potential weaknesses."
            ),
            # Add more offensive measures here
        }

    @staticmethod
    def get_defensive_measures() -> Dict[str, SecurityMeasure]:
        return {
            "waf_shield": SecurityMeasure(
                name="WAF Shield",
                strategy=SecurityStrategy.DEFENSIVE,
                cost=25,
                effectiveness=20,
                setup_time=1,
                description="Deploy a Web Application Firewall to protect against web attacks."
            ),
            "security_groups": SecurityMeasure(
                name="Security Groups",
                strategy=SecurityStrategy.DEFENSIVE,
                cost=10,
                effectiveness=15,
                setup_time=1,
                description="Configure network access controls via security groups."
            ),
            "encryption": SecurityMeasure(
                name="Data Encryption",
                strategy=SecurityStrategy.DEFENSIVE,
                cost=20,
                effectiveness=25,
                setup_time=2,
                description="Implement encryption for data protection."
            ),
            # Add more defensive measures here
        }
```

**Integration Steps:**

1. **Create a New File:** Save the above code in a new file named `security_measures.py`.
2. **Import in `vpc_defender.py`:**

    ```python
    from security_measures import SecurityMeasures, SecurityStrategy, SecurityMeasure
    ```

---

### **b. Define Threats**

Ensure that threats are properly defined and can be generated based on regions.

```python
# threat_generator.py

from threat import Threat
from typing import Dict

def generate_threat_for_region(region: str) -> Threat:
    """
    Generates a threat based on the region.
    """
    region_threats = {
        "US East (N. Virginia)": Threat(
            name="DDoS Attack",
            attack_type="Network Flood",
            power=25,
            persistence=3,
            adaptability=60
        ),
        "US West (Oregon)": Threat(
            name="Data Breach",
            attack_type="Unauthorized Access",
            power=30,
            persistence=4,
            adaptability=70
        ),
        "Special Region": Threat(
            name="Advanced Threat",
            attack_type="Multi-vector Attack",
            power=35,
            persistence=5,
            adaptability=80
        ),
        # Add more regions and corresponding threats
    }
    return region_threats.get(region, Threat(
        name="Generic Threat",
        attack_type="Unknown",
        power=20,
        persistence=2,
        adaptability=50
    ))
```

**Integration Steps:**

1. **Create a New File:** Save the above code in a new file named `threat_generator.py`.
2. **Ensure `Threat` Class is Defined:**

    ```python
    # threat.py

    from dataclasses import dataclass

    @dataclass
    class Threat:
        name: str
        attack_type: str
        power: int
        persistence: int
        adaptability: int

        def calculate_damage(self, defender: 'VPCDefender') -> int:
            """
            Calculates the damage inflicted by the threat on the defender.
            """
            base_damage = random.randint(self.power - 5, self.power + 5)
            # Apply defender's defensive measures
            for measure in defender.active_measures:
                if measure.strategy == SecurityStrategy.DEFENSIVE:
                    base_damage = max(0, base_damage - measure.effectiveness)
            # Apply defender's stats for further damage reduction
            damage_reduction = (defender.stats.fault_tolerance + defender.stats.resilience) / 100
            final_damage = int(base_damage * (1 - damage_reduction))
            return final_damage
    ```

3. **Import in `game.py`:**

    ```python
    from threat_generator import generate_threat_for_region
    ```

---

### **c. Define the `SecurityBattle` Class**

Ensure that the `SecurityBattle` class handles combat between the defender and the threat.

```python
# security_battle.py

class SecurityBattle:
    """
    Manages the battle between the VPC Defender and a Threat.
    """
    def __init__(self, defender: VPCDefender, threat: Threat):
        self.defender = defender
        self.threat = threat
        self.round = 1

    def start_battle(self):
        """
        Starts the battle until either the defender or the threat is defeated.
        """
        print(f"\nA threat '{self.threat.name}' has emerged! Preparing for battle...")
        while self.defender.is_alive() and self.threat.persistence > 0:
            print(f"\n--- Round {self.round} ---")
            self.player_turn()
            if not self.threat_defeated():
                self.threat_turn()
            self.round += 1
        if self.defender.is_alive() and self.threat.persistence <= 0:
            print("Threat has been neutralized!")
        elif not self.defender.is_alive():
            print("Defender has been defeated!")

    def player_turn(self):
        """
        Handles the defender's actions during their turn.
        """
        action = input("Do you want to 'attack', 'cast spell', or 'use item'? ").strip().lower()
        if action == 'attack':
            offensive_power = sum(
                measure.effectiveness for measure in self.defender.active_measures
                if measure.strategy == SecurityStrategy.OFFENSIVE
            )
            counter_damage = int(offensive_power * (1 + self.defender.stats.agility / 100))
            self.threat.power -= counter_damage
            print(f"Attacked the threat! Dealt {counter_damage} damage.")
        elif action == 'cast spell':
            print("Available Spells:")
            for key, spell in self.defender.spells.items():
                print(f"- {key}: {spell.description}")
            spell_key = input("Enter the spell key to cast: ").strip().lower()
            result = self.defender.cast_spell(spell_key)
            print(result)
        elif action == 'use item':
            # Since inventory is removed, inform the player
            print("No items available to use.")
        else:
            print("Invalid action. Skipping turn.")
    
    def threat_turn(self):
        """
        Handles the threat's actions during its turn.
        """
        damage = self.threat.calculate_damage(self.defender)
        is_defeated = self.defender.take_damage(damage)
        if is_defeated:
            print("The threat has defeated your defender!")
        else:
            print(f"The threat dealt {damage} damage to your defender.")
    
    def threat_defeated(self) -> bool:
        """
        Checks if the threat has been defeated.
        """
        return self.threat.power <= 0
```

**Integration Steps:**

1. **Create a New File:** Save the above code in a new file named `security_battle.py`.
2. **Import `SecurityBattle`:**

    ```python
    from security_battle import SecurityBattle
    ```


---

## **7. Summary of File Structure**

For clarity, here's how your project files should be organized:

```
your_project/
│
├── game.py
├── game_board.py
├── player.py
├── vpc_defender.py
├── spells.py
├── security_measures.py
├── threat.py
├── threat_generator.py
└── security_battle.py
```


## **8. Testing the Integrated Features**

### **a. Running the Game**

1. **Ensure All Files are Saved:** Make sure all the above files (`game.py`, `game_board.py`, etc.) are saved in the same directory.

2. **Run the Game:**
   - Open your terminal or command prompt.
   - Navigate to the project directory.
   - Execute the game using:

     ```bash
     python game.py
     ```

### **b. Sample Interaction**

Here's how a sample game session might look:

```
=== Welcome to the Cloud Security Battle ===
Player1's home region is US East (N. Virginia) - us-east-1a.

=== New Turn ===
Do you want to 'roll' the die or 'draw' a card to move? (roll/draw): roll
You rolled a 4.
Player1 landed on US West (Oregon) - us-west-2b.
Operating in US West (Oregon) incurs a charge of 10 preparation points.

--- Setup Phase ---
Preparation Points: 90
Available Defensive Measures:
- waf_shield: Deploy a Web Application Firewall to protect against web attacks. (Cost: 25, Effectiveness: 20)
- security_groups: Configure network access controls via security groups. (Cost: 10, Effectiveness: 15)
- encryption: Implement encryption for data protection. (Cost: 20, Effectiveness: 25)
Available Offensive Measures:
- penetration_test: Proactively identify vulnerabilities through penetration testing. (Cost: 30, Effectiveness: 25)
- security_scanning: Scan infrastructure for potential weaknesses. (Cost: 15, Effectiveness: 20)
Enter 'measure' to implement a security measure or 'done' to finish setup: measure
Enter the measure key to implement: security_groups
Implemented Security Groups with effectiveness bonus of 15.

Enter 'measure' to implement a security measure or 'done' to finish setup: done
Setup phase completed.

--- Threat Encounter ---
A threat 'Data Breach' has emerged! Preparing for battle...

--- Round 1 ---
Do you want to 'attack', 'cast spell', or 'use item'? attack
Attacked the threat! Dealt 25 damage.
The threat dealt 15 damage to your defender.

--- Round 2 ---
Do you want to 'attack', 'cast spell', or 'use item'? cast spell
Available Spells:
- lambda_edge: Run code closer to users to increase agility.
- cloudfront: Accelerate content delivery to improve elasticity.
Enter the spell key to cast: lambda_edge
AWS Lambda@Edge cast! Agility increased by 12.
Cast spell: AWS Lambda@Edge.

The threat dealt 10 damage to your defender.

--- Round 3 ---
Do you want to 'attack', 'cast spell', or 'use item'? attack
Attacked the threat! Dealt 15 damage.
Threat has been neutralized!
Successfully defeated the threat: Data Breach!

CloudGuardian has leveled up to Level 2!
Stats have been improved and health restored.

Do you want to continue to the next turn? (yes/no): no
Thank you for playing!
```

**Explanation of Sample Interaction:**

1. **Home Region Setup:** The player is assigned a home region.
2. **Movement:** The player chooses to roll a die and moves to a new region, incurring charges for operating outside the home region.
3. **Setup Phase:** The player implements security measures, spending preparation points.
4. **Threat Encounter:** A region-specific threat appears, and the battle begins.
5. **Combat Rounds:** The player can choose to attack or cast spells to deal with the threat.
6. **Level Up:** Upon defeating the threat, the defender levels up, enhancing stats.
7. **Game Continuation:** The player can choose to continue or end the game.

---

## **9. Final Touches and Recommendations**

### **a. Balancing the Game**

- **Adjust Costs and Effectiveness:**
  - Ensure that the costs of security measures are balanced with their effectiveness.
  - Higher-level threats should require more robust defenses.

- **Resource Management:**
  - Carefully manage preparation points and charges to maintain game difficulty.

- **Spell Usage:**
  - Limit the number of spells that can be cast per battle to prevent overpowering defenses.

### **b. Enhancing User Experience**

- **Clear Instructions:**
  - Provide clear prompts and feedback to guide players through their choices.

- **Combat Feedback:**
  - Offer detailed feedback during battles to inform players of the impact of their actions.

- **Replayability:**
  - Introduce randomness in threats and spells to enhance replayability.

### **c. Expanding the Game**

- **Additional Regions and AZs:**
  - Add more regions and AZs with unique bonuses and threats.

- **More Spells and Measures:**
  - Introduce a wider variety of spells and security measures for strategic depth.

- **Storyline and Quests:**
  - Develop a storyline with quests and objectives to engage players.

- **Multiple Players:**
  - Consider adding multiplayer support for competitive or cooperative gameplay.

---

## **Conclusion**

By following the above steps and integrating the provided code snippets, you can enhance your game with a robust movement system, serverless spells, and dynamic threat encounters. This modular approach ensures that you can continue to expand and refine your game without the need to overhaul your existing codebase.

Feel free to adjust the mechanics, costs, and effects to best fit the gameplay experience you envision. If you encounter any issues or need further assistance with specific integrations or features, don't hesitate to ask!